In [ ]:
!pip install -q --user datascience                   
!pip install -q --user pandas-profiling              
!pip install -q --user yellowbrick                   
!pip install mysql-connector-python
!pip install googletrans==3.1.0a0
!pip install deep-translator

In [15]:
#!pip install -q --upgrade pandas-profiling
#!pip install -q --upgrade yellowbrick

In [16]:
#-------------------------------------------------------------------------------------------------------------------------------
import pandas as pd                                                 # Importing for panel data analysis
from pandas_profiling import ProfileReport                          # Import Pandas Profiling (To generate Univariate Analysis) 
pd.set_option('display.max_columns', None)                          # Unfolding hidden features if the cardinality is high      
pd.set_option('display.max_colwidth', None)                         # Unfolding the max feature width for better clearity      
pd.set_option('display.max_rows', None)                             # Unfolding hidden data points if the cardinality is high
pd.set_option('mode.chained_assignment', None)                      # Removing restriction over chained assignments operations
pd.set_option('display.float_format', lambda x: '%.5f' % x)         # To suppress scientific notation over exponential values
#-------------------------------------------------------------------------------------------------------------------------------
import numpy as np                                                  # Importing package numpys (For Numerical Python)
from scipy.stats import randint as sp_randint                       # for initializing random integer values
#-------------------------------------------------------------------------------------------------------------------------------
import matplotlib.pyplot as plt                                     # Importing pyplot interface using matplotlib
from matplotlib.pylab import rcParams                               # Backend used for rendering and GUI integration                                               
import seaborn as sns                                               # Importin seaborm library for interactive visualization
%matplotlib inline
#-------------------------------------------------------------------------------------------------------------------------------
from sklearn.metrics import accuracy_score                          # For calculating the accuracy for the model
from sklearn.metrics import precision_score                         # For calculating the Precision of the model
from sklearn.metrics import recall_score                            # For calculating the recall of the model
from sklearn.metrics import precision_recall_curve                  # For precision and recall metric estimation
from sklearn.metrics import confusion_matrix                        # For verifying model performance using confusion matrix
from sklearn.metrics import f1_score                                # For Checking the F1-Score of our model  
from sklearn.metrics import roc_curve                               # For Roc-Auc metric estimation
#-------------------------------------------------------------------------------------------------------------------------------
from sklearn.model_selection import train_test_split                # To split the data in training and testing part     
from sklearn.feature_selection import SelectFromModel               # To perform Feature Selection over model

#-------------------------------------------------------------------------------------------------------------------------------
import warnings                                                     # Importing warning to disable runtime warnings
warnings.filterwarnings("ignore")                                   # Warnings will appear only once
#-------------------------------------------------------------------------------------------------------------------------------
import mysql.connector as connection
import googletrans
from googletrans import Translator
from deep_translator import GoogleTranslator

Importing 1st dataset - 'events_data' from a csv file


In [38]:
# Reading the data from the events data csv file
df_events_data = pd.read_csv('events_data.csv')
df_events_data.head()

,event_id,device_id,timestamp,longitude,latitude,city,state
0,2765368,2973347786994914304.00000,2016-05-07 22:52:05,77.22568,28.73014,Delhi,Delhi
1,2955066,4734221357723754496.00000,2016-05-01 20:44:16,88.38836,22.66033,Calcutta,WestBengal
2,605968,-3264499652692492800.00000,2016-05-02 14:23:04,77.25681,28.75791,Delhi,Delhi
3,448114,5731369272434022400.00000,2016-05-03 13:21:16,80.34361,13.15333,Chennai,TamilNadu
4,665740,338888002570799936.00000,2016-05-06 03:51:05,85.99774,23.84261,Bokaro,Jharkhand


In [39]:
#checking the shape of the dataset
df_events_data.shape

(3252950, 7)

In [40]:
df_events_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3252950 entries, 0 to 3252949
Data columns (total 7 columns):
 #   Column     Dtype  
---  ------     -----  
 0   event_id   int64  
 1   device_id  float64
 2   timestamp  object 
 3   longitude  float64
 4   latitude   float64
 5   city       object 
 6   state      object 
dtypes: float64(3), int64(1), object(3)
memory usage: 173.7+ MB


In [41]:
#Checking if null values are present in events dataset
df_events_data.isnull().sum()

event_id       0
device_id    453
timestamp      0
longitude    423
latitude     423
city           0
state        377
dtype: int64

 **Importing 2nd & 3rd dataset - 'gender_age_train' & 'phone_brand_device_model' from MySQL database:**

1.gender_age_train :- Device_ids and their respective user gender, age and age_group

2.phone_brand_device_model :- device ids, brand, and device's model.

In [17]:
#Downloading the data from the MySQL database for gender_age_train & phone_brand_device_model onto Python by connecting to the below provided MySQL instance.
try:
    mydb = connection.connect(host="cpanel.insaid.co", database = 'Capstone1',user="student", passwd="student",use_pure=True)
    query1 = "Select * from gender_age_train;"
    query2 = "Select * from phone_brand_device_model;"
    df_gender_age = pd.read_sql(query1,mydb)
    df_brand_model = pd.read_sql(query2,mydb)
    mydb.close() #close the connection
except Exception as e:
    mydb.close()
    print(str(e))

**Getting the head for the gender_age dataset**

In [18]:
df_gender_age.head()

,device_id,gender,age,group
0,-8076087639492063270,M,35,M32-38
1,-2897161552818060146,M,35,M32-38
2,-8260683887967679142,M,35,M32-38
3,-4938849341048082022,M,30,M29-31
4,245133531816851882,M,30,M29-31


In [19]:
df_gender_age.shape

(74645, 4)

In [28]:
df_gender_age.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74645 entries, 0 to 74644
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   device_id  74645 non-null  int64 
 1   gender     74645 non-null  object
 2   age        74645 non-null  int64 
 3   group      74645 non-null  object
dtypes: int64(2), object(2)
memory usage: 2.3+ MB


In [80]:
df_gender_age.describe()

,device_id,age
count,74645.00000,74645.00000
mean,-749135388419837.00000,31.41034
std,5327149733911456768.00000,9.86873
min,-9223067244542181376.00000,1.00000
25%,-4617366812584264704.00000,25.00000
50%,-18413620249632024.00000,29.00000
75%,4636655899093149696.00000,36.00000
max,9222849349208140800.00000,96.00000


In [37]:
df_gender_age['group']

0        M32-38
1        M32-38
2        M32-38
3        M29-31
4        M29-31
5        F24-26
6        M32-38
7        M32-38
8        M32-38
9        F33-42
10       M29-31
11       F33-42
12       F27-28
13       M32-38
14         M39+
15       F27-28
16       M32-38
17       M32-38
18         M39+
19       M32-38
20       F33-42
21       M23-26
22       M23-26
23       M27-28
24       M29-31
25       M32-38
26       M32-38
27       F33-42
28       M32-38
29       M32-38
30       M32-38
31       M32-38
32         M22-
33       M29-31
34       M32-38
35       M32-38
36       M23-26
37       M32-38
38       M32-38
39       M29-31
40       M27-28
41         M22-
42       M29-31
43       M29-31
44       M27-28
45       M27-28
46       M27-28
47       M27-28
48       F27-28
49       M29-31
50       F33-42
51       M29-31
52       M29-31
53       F33-42
54       F33-42
55       M32-38
56       M32-38
57       M32-38
58         M39+
59       M29-31
60       M23-26
61       M23-26
62      

**Getting the head for the brand_model dataset**

In [20]:
df_brand_model.head()

,device_id,phone_brand,device_model
0,1877775838486905855,vivo,Y13
1,-3766087376657242966,小米,V183
2,-6238937574958215831,OPPO,R7s
3,8973197758510677470,三星,A368t
4,-2015528097870762664,小米,红米Note2


In [21]:
df_brand_model.shape

(87726, 3)

In [76]:
df_brand_model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87726 entries, 0 to 87725
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   device_id             87726 non-null  int64 
 1   phone_brand           87726 non-null  object
 2   device_model          87726 non-null  object
 3   phone_brand_english   87726 non-null  object
 4   device_model_english  87726 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.3+ MB


In [22]:
g_translator = GoogleTranslator()


#df_brand_model['phone_brand_english'] = df_brand_model['phone_brand'].apply(translator.translate).apply(getattr, args=('text',))

df_brand_model['phone_brand_english']= df_brand_model['phone_brand']
df_brand_model['device_model_english']= df_brand_model['device_model']

# Translating phone_brand
brand_translations = {}
brand_unique_elements = df_brand_model['phone_brand'].unique()
for brand_element in brand_unique_elements:
# add translation to the dictionary
    brand_translations[brand_element] = g_translator.translate(brand_element, src='zh-cn')    
df_brand_model['phone_brand_english'].replace(brand_translations, inplace = True)
df_brand_model['phone_brand_english'].unique()

array(['vivo', 'Millet', 'OPPO', 'Samsung', 'Coolpad', 'Lenovo', 'Huawei',
       'cool', 'Meizu', 'PHICOMM', 'China Mobile', 'HTC', 'Amagatarai',
       'Supreme Treasure', 'LG', 'Obersin', 'Quality rice', 'YOU',
       'Nubia', 'HP', 'Nibiru', 'Meitu', 'rural rice', 'Motorola',
       'dream rice', 'hammer', 'InFocus', 'LeTV', 'Hisense', 'Bailifeng',
       'one plus', 'language letter', 'Haier', 'Cool than', 'Newman',
       'waveguide', 'Dove', 'listen to the rhyme', 'TCL', 'Cruel',
       'Apple', 'LOGO', 'shallots', 'fruit rice', 'ASUS', 'Onda', 'Ayuni',
       'Konka', 'Yougo', 'Banghua', 'Cyber \u200b\u200bYuhua',
       'black rice', 'Lovme', 'pioneer', 'E Pie', 'Shenzhou', 'Nokia',
       'Purnell', 'candied haws', 'Atsu', 'Ou Xin', 'Mickey',
       "Cube Rubik's Cube", 'Blue demon', 'small poplar', 'Belfon',
       'Sticky rice', 'Mi Ge', 'E person E book', 'Simi', 'Big Q',
       'Taipower', 'Philips', 'Weimi', 'Daxian', 'Changhong',
       'Dimensional', 'green orange', 't

In [25]:
import re
from datetime import datetime

# Translating device_model
device_model_element_translations = {}
device_model_unique_elements = df_brand_model['device_model'].unique()
print(datetime.now())
for device_model_element in device_model_unique_elements:    
# add translation to the dictionary
#Checking for string containing only numbers as they are throwing error, so ignoring them    
    if(re.match(r"[-+]?\d+(\.0*)?$", device_model_element) is None):
        device_model_element_translations[device_model_element] = g_translator.translate(device_model_element)    
df_brand_model['device_model_english'].replace(device_model_element_translations, inplace = True)
df_brand_model['device_model_english'].unique()
print(datetime.now())

2022-02-26 13:08:31.640130
2022-02-26 13:10:59.040722


In [27]:
df_brand_model.sample(50)

,device_id,phone_brand,device_model,phone_brand_english,device_model_english
30931,-6976109185992643294,努比亚,Z7 Mini,Nubia,Z7 Mini
65163,808959373111076115,小米,红米,Millet,Red rice
19267,6447755340384224994,HTC,M8St,HTC,M8St
80819,1892006536093729851,华为,荣耀畅玩5,Huawei,Honor Play 5
26173,-207934154262067872,酷派,大神F2,Coolpad,Okami F2
83300,-28694643595904273,华为,Ascend P7,Huawei,Ascend P7
46113,5402317977964869783,小米,MI 4,Millet,MI 4
75940,-3188049036922581785,vivo,X6 Plus D,vivo,X6 Plus D
59497,8949748515875643237,华为,荣耀6,Huawei,glory 6
77314,7923923849056503030,华为,荣耀3C,Huawei,Honor 3C


Filtering the events dataset by states (WestBengal, Karnataka, Bihar, Punjab,Gujarat, Kerala)
For consulting, team is to focus on 6 states: WestBengal, Karnataka, Bihar, Punjab,Gujarat and Kerala.
We observed that Events dataset has 377 missing values in 'state' column.
So, first we have to handle these missing values before we filter the dataset by states.

In [42]:
df_events_data[df_events_data['state'].isnull()].sample(5)

,event_id,device_id,timestamp,longitude,latitude,city,state
2476333,1397137,-1752057305775059712.00000,2016-05-03 20:56:43,75.93097,22.81884,Indore,NaN
481271,2381514,-2366408909213987328.00000,2016-05-03 23:12:53,83.37301,17.77542,Visakhapatnam,NaN
16243,1918653,3137053756433768448.00000,2016-05-03 00:46:10,83.32671,17.82975,Visakhapatnam,NaN
247532,151113,-5829328361235250176.00000,2016-05-03 13:17:55,73.85167,18.59331,Pune,NaN
1513978,904996,-5858653205052383232.00000,2016-05-07 15:20:54,88.61942,25.48446,Gangarampur,NaN


**Handling the missing value in 'state' column for WestBengal, Karnataka, Bihar, Punjab,Gujarat, Kerala**

In [44]:
#finding the unique states
df_events_data['state'].unique()

array(['Delhi', 'WestBengal', 'TamilNadu', 'Jharkhand', 'AndhraPradesh',
       'Maharashtra', 'Gujarat', 'Kerala', 'MadhyaPradesh', 'Karnataka',
       'Rajasthan', 'Orissa', 'Punjab', 'UttarPradesh', 'Nagaland',
       'Haryana', 'Telangana', 'Chhattisgarh', 'Bihar', 'JammuandKashmir',
       'Assam', 'Goa', 'Mizoram', 'Tripura', 'Uttaranchal', nan,
       'Pondicherry', 'Manipur', 'Meghalaya', 'ArunachalPradesh',
       'HimachalPradesh', 'Chandigarh', 'AndamanandNicobarIslands'],
      dtype=object)

In [45]:
#finding out the cities for the missing state so that we can fill the missing states from the corresponding city.
(df_events_data['city'][df_events_data['state'].isnull()]).unique()

array(['Pune', 'Visakhapatnam', 'Indore', 'Chennai', 'Delhi',
       'Channapatna', 'Jaipur', 'Gangarampur', 'Arambagh'], dtype=object)

**We will fill the missing states for these 3 cities:**

Channapatna -> Karnataka

Gangarampur -> WestBengal

Arambagh -> WestBengal

In [46]:
#Replacing the nan values in state with Karnataka where city is Channapatna
df_events_data.loc[(df_events_data['city'] == 'Channapatna') & (df_events_data['state'].isnull()), 'state'] = 'Karnataka'

In [47]:
#Replacing the nan values in state with WestBengal where city is Gangarampur
df_events_data.loc[(df_events_data['city'] == 'Gangarampur') & (df_events_data['state'].isnull()), 'state'] = 'WestBengal'


In [48]:
#Replacing the nan values in state with WestBengal where city is Arambagh
df_events_data.loc[(df_events_data['city'] == 'Arambagh') & (df_events_data['state'].isnull()), 'state'] = 'WestBengal'


In [49]:
#Re-checking if events dataset contains any missing states for WestBengal, Karnataka, Bihar, Punjab,Gujarat and Kerala 
#by checking the list of cities for the missing states.
(df_events_data['city'][df_events_data['state'].isnull()]).unique()

array(['Pune', 'Visakhapatnam', 'Indore', 'Chennai', 'Delhi', 'Jaipur'],
      dtype=object)

In [50]:
df_events_data['state'].isnull().sum()

321

Only 56 missing states belonged to WestBengal, Karnataka, Bihar, Punjab,Gujarat, Kerala.

Filtering the events database by states (WestBengal, Karnataka, Bihar, Punjab,Gujarat, Kerala)

In [51]:
df_events_data_filtered = df_events_data[df_events_data['state'].isin(['WestBengal', 'Karnataka', 'Bihar', 'Punjab','Gujarat', 'Kerala'])]

In [52]:
df_events_data_filtered.head()

,event_id,device_id,timestamp,longitude,latitude,city,state
1,2955066,4734221357723754496.00000,2016-05-01 20:44:16,88.38836,22.66033,Calcutta,WestBengal
28,769546,-1817502319478569728.00000,2016-05-01 14:07:23,88.37181,22.66285,Calcutta,WestBengal
30,1750603,-5598137337131307008.00000,2016-05-05 15:47:03,70.21268,23.11837,Gandhidham,Gujarat
31,3085968,-3808296883972395520.00000,2016-05-07 01:25:47,75.51302,11.81237,Thalassery,Kerala
39,1407594,-2995507760806350336.00000,2016-05-03 20:01:35,77.80519,13.53330,ChikBallapur,Karnataka


In [53]:
df_events_data_filtered.describe()

,event_id,device_id,longitude,latitude
count,422971.00000,422923.00000,422929.00000,422929.00000
mean,1635553.21051,54647204513961192.00000,82.33530,20.79314
std,930915.34686,5330500297691951104.00000,6.38155,5.42450
min,20881.00000,-9221066489596332032.00000,12.56740,8.41244
25%,822921.00000,-4582571963008606720.00000,76.55060,15.20491
50%,1632956.00000,124460672713006880.00000,85.51903,22.62057
75%,2443386.00000,4703286051045099520.00000,88.40805,23.10415
max,3252946.00000,9222849349208140800.00000,89.62207,41.87190


In [54]:
df_events_data_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 422971 entries, 1 to 3252921
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   event_id   422971 non-null  int64  
 1   device_id  422923 non-null  float64
 2   timestamp  422971 non-null  object 
 3   longitude  422929 non-null  float64
 4   latitude   422929 non-null  float64
 5   city       422971 non-null  object 
 6   state      422971 non-null  object 
dtypes: float64(3), int64(1), object(3)
memory usage: 25.8+ MB


In [55]:
#checking null values in the filtered dataset
df_events_data_filtered.isnull().sum()

event_id      0
device_id    48
timestamp     0
longitude    42
latitude     42
city          0
state         0
dtype: int64

**Observations for the filtered event dataset:**

There are 422971 records and 7 features in the filtered events dataset.

event_id, timestamp, city and state columns have no missing values.

device_id has 48 missing values.

longitude and latitude have 42 missing values.

Data types for all the columns are correct.

**Merging the filtered events dataset with gender_age and brand_model datasets.**
Here, we will map all the records of events dataset with _genderage and _brandmodel datasets to get the demographic details and the brand model details of the users.

We will create a new merged dataframe which will be used for further analysis.
Merging of the datasets will happen on the common column device id.

So, first we need to fill in the missing values for the device id in the filtered events dataset.

In [56]:
df_events_data_filtered[df_events_data_filtered['device_id'].isnull()].groupby(['longitude', 'latitude','city','state']).count() #sort_values(['state','city'])


,,,,event_id,device_id,timestamp
longitude,latitude,city,state,,,
70.68639,21.79069,Jetpur,Gujarat,16,0,16
73.16934,21.19428,Bardoli,Gujarat,16,0,16
75.99255,31.56175,Hoshiarpur,Punjab,16,0,16


We see that there are 3 device_ids whose values are missing:

They can be mapped as:
1st device id where longitude = 73.16934, latitude = 21.19428, city= Bardoli , state=Gujarat

2nd device id where longitude = 70.68639, latitude = 21.79069, city= Jetpur, state=Gujarat

3rd device id where longitude = 75.99255, latitude = 31.56175, city= Hoshiarpur, state=Punjab

In [66]:
# To get the 1st device id where longitude = 73.16934, latitude = 21.19428, city= Bardoli , state=Gujarat
df_events_data_filtered[(df_events_data_filtered['city'] == 'Bardoli') & (df_events_data_filtered['state'] == 'Gujarat')].groupby(['device_id','longitude', 'latitude','city','state']).count()


,,,,,event_id,timestamp
device_id,longitude,latitude,city,state,,
-8939375088504041472.00000,73.20107,21.17746,Bardoli,Gujarat,15,15
-8245807954961570816.00000,73.15158,21.19418,Bardoli,Gujarat,1,1
-8215770519233685504.00000,73.16934,21.19428,Bardoli,Gujarat,381,381
-8101478099587991552.00000,73.16179,21.20450,Bardoli,Gujarat,5,5
-8022320605308700672.00000,73.20153,21.19053,Bardoli,Gujarat,18,18
-7629548547563755520.00000,73.20758,21.15599,Bardoli,Gujarat,8,8
-7046882659286756352.00000,73.21763,21.14622,Bardoli,Gujarat,1,1
-6624996692128278528.00000,73.15566,21.18198,Bardoli,Gujarat,13,13
-6313122852748910592.00000,73.13923,21.20802,Bardoli,Gujarat,84,84


1st device id is -8215770519233685504 where longitude = 73.16934, latitude = 21.19428, city= Bardoli , state=Gujarat

In [67]:
# To get the 2nd device id where longitude = 70.68639, latitude = 21.79069, city= Jetpur, state=Gujarat
df_events_data_filtered[(df_events_data_filtered['city'] == 'Jetpur') & (df_events_data_filtered['state'] == 'Gujarat')].groupby(['device_id','longitude', 'latitude','city','state']).count()


,,,,,event_id,timestamp
device_id,longitude,latitude,city,state,,
-9100626844458295296.00000,70.68467,21.79341,Jetpur,Gujarat,32,32
-7864383677585044480.00000,70.62907,21.82055,Jetpur,Gujarat,95,95
-7356791296642819072.00000,70.68620,21.80623,Jetpur,Gujarat,1,1
-6941872424033244160.00000,70.62482,21.80541,Jetpur,Gujarat,4,4
-4748170714083747840.00000,70.62804,21.76449,Jetpur,Gujarat,9,9
-4619283383535598592.00000,70.68497,21.78174,Jetpur,Gujarat,17,17
-4161076139992630784.00000,70.63390,21.82504,Jetpur,Gujarat,26,26
-4109861778213652480.00000,70.64444,21.81232,Jetpur,Gujarat,2,2
-3642392870862093312.00000,70.67585,21.82392,Jetpur,Gujarat,22,22


2nd device id is -1688015122502424064 where longitude = 70.68639, latitude = 21.79069, city= Jetpur, state=Gujarat

In [68]:
# To get the 3rd device id where longitude = 75.99255, latitude = 31.56175, city= Hoshiarpur, state=Punjab
df_events_data_filtered[(df_events_data_filtered['city'] == 'Hoshiarpur') & (df_events_data_filtered['state'] == 'Punjab')].groupby(['device_id','longitude', 'latitude','city','state']).count()


,,,,,event_id,timestamp
device_id,longitude,latitude,city,state,,
-9179704438540888064.00000,76.00749,31.56383,Hoshiarpur,Punjab,5,5
-9003299201851031552.00000,75.96806,31.57004,Hoshiarpur,Punjab,1,1
-7959352580287944704.00000,75.97433,31.60916,Hoshiarpur,Punjab,12,12
-7516713034711112704.00000,75.92446,31.60512,Hoshiarpur,Punjab,16,16
-6314072935617405952.00000,75.93961,31.55397,Hoshiarpur,Punjab,31,31
-6016575845341766656.00000,75.95764,31.60668,Hoshiarpur,Punjab,11,11
-5583822054383365120.00000,75.95360,31.60232,Hoshiarpur,Punjab,2,2
-4252602101177080320.00000,75.99204,31.62383,Hoshiarpur,Punjab,10,10
-3923551471103946240.00000,75.96350,31.61652,Hoshiarpur,Punjab,113,113


3rd device id is 1750778632182066944 where longitude = 75.99255, latitude = 31.56175, city= Hoshiarpur, state=Punjab

**Filling the missing values for device ids**

In [69]:
#Replacing the missing device id for 1st missing device
df_events_data_filtered.loc[((df_events_data_filtered['city'] == 'Bardoli') & (df_events_data_filtered['state'] == 'Gujarat')) & (df_events_data['device_id'].isnull()), 'device_id'] = -8215770519233685504


In [70]:
#Replacing the missing device id for 2nd missing device
df_events_data_filtered.loc[((df_events_data_filtered['city'] == 'Jetpur') & (df_events_data_filtered['state'] == 'Gujarat')) & (df_events_data['device_id'].isnull()), 'device_id'] = -1688015122502424064


In [71]:
#Replacing the missing device id for 3rd missing device
df_events_data_filtered.loc[((df_events_data_filtered['city'] == 'Hoshiarpur') & (df_events_data_filtered['state'] == 'Punjab')) & (df_events_data['device_id'].isnull()), 'device_id'] = 1750778632182066944


In [72]:
df_events_data_filtered.isnull().sum()

event_id      0
device_id     0
timestamp     0
longitude    42
latitude     42
city          0
state         0
dtype: int64

In [73]:
df_events_data_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 422971 entries, 1 to 3252921
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   event_id   422971 non-null  int64  
 1   device_id  422971 non-null  float64
 2   timestamp  422971 non-null  object 
 3   longitude  422929 non-null  float64
 4   latitude   422929 non-null  float64
 5   city       422971 non-null  object 
 6   state      422971 non-null  object 
dtypes: float64(3), int64(1), object(3)
memory usage: 41.9+ MB


1st Merging the filtered event dataset with gender_age dataset on _deviceid.

First, we need to convert the data type of device_id in gender_age dataset to float.

In [74]:
df_gender_age['device_id'] = df_gender_age['device_id'].astype(df_events_data_filtered['device_id'].dtype)

In [75]:
df_gender_age.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74645 entries, 0 to 74644
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   device_id  74645 non-null  float64
 1   gender     74645 non-null  object 
 2   age        74645 non-null  int64  
 3   group      74645 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 2.3+ MB


In [81]:
df_events_gender = pd.merge(df_events_data_filtered, df_gender_age, on='device_id', how='left')
df_events_gender.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 422971 entries, 0 to 422970
Data columns (total 10 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   event_id   422971 non-null  int64  
 1   device_id  422971 non-null  float64
 2   timestamp  422971 non-null  object 
 3   longitude  422929 non-null  float64
 4   latitude   422929 non-null  float64
 5   city       422971 non-null  object 
 6   state      422971 non-null  object 
 7   gender     422971 non-null  object 
 8   age        422971 non-null  int64  
 9   group      422971 non-null  object 
dtypes: float64(3), int64(2), object(5)
memory usage: 35.5+ MB


In [82]:
df_events_gender.isnull().sum()

event_id      0
device_id     0
timestamp     0
longitude    42
latitude     42
city          0
state         0
gender        0
age           0
group         0
dtype: int64

In [84]:
df_events_gender.describe()

,event_id,device_id,longitude,latitude,age
count,422971.00000,422971.00000,422929.00000,422929.00000,422971.00000
mean,1635553.21051,54332593871695992.00000,82.33530,20.79314,31.44556
std,930915.34686,5330461509042279424.00000,6.38155,5.42450,9.78804
min,20881.00000,-9221066489596332032.00000,12.56740,8.41244,11.00000
25%,822921.00000,-4582571963008606720.00000,76.55060,15.20491,25.00000
50%,1632956.00000,124460672713006880.00000,85.51903,22.62057,29.00000
75%,2443386.00000,4703286051045099520.00000,88.40805,23.10415,36.00000
max,3252946.00000,9222849349208140800.00000,89.62207,41.87190,88.00000


In [89]:
df_events_gender.sample(500)

,event_id,device_id,timestamp,longitude,latitude,city,state,gender,age,group
392537,1892058,-4876619845963159552.00000,2016-05-06 10:03:21,74.86628,30.67017,KotKapura,Punjab,M,28,M27-28
361450,2780441,2398645792123232768.00000,2016-05-02 14:02:35,77.59453,13.02485,Bangalore,Karnataka,F,27,F27-28
76791,727803,3084418677851400192.00000,2016-05-01 08:24:53,88.17558,25.65518,Raiganj,WestBengal,M,24,M23-26
234607,1068924,8194773058374627328.00000,2016-05-04 15:59:40,72.19239,24.29324,Disa,Gujarat,M,26,M23-26
376146,2341380,-4771188762662166528.00000,2016-05-03 14:59:53,76.74334,10.84994,Palakkad,Kerala,F,27,F27-28
341231,1741558,-6536965139434339328.00000,2016-05-06 08:32:05,88.45123,22.71720,Panihati,WestBengal,M,19,M22-
301031,1822677,3812824902750585344.00000,2016-05-04 09:20:57,75.80974,13.34846,Chikmagalur,Karnataka,M,24,M23-26
261499,2255200,1899732562558298880.00000,2016-05-01 03:08:17,75.94024,10.95936,Tirur,Kerala,F,21,F23-
201796,998145,-5308446867342072832.00000,2016-05-03 21:27:14,76.98072,13.76935,Sira,Karnataka,F,19,F23-
232726,1847528,5611636094222514176.00000,2016-05-04 08:06:37,77.59145,13.05877,Bangalore,Karnataka,M,43,M39+
